# Take the Tour

_(The tour is auto-generated from 
[this jupyter notebook](https://github.com/mar10/nutree/blob/main/docs/jupyter/take_the_tour.ipynb).)_

Nutree organizes arbitrary object instances in an unobtrusive way. <br>
That means, we can add existing objects without having to derive from a common 
base class or implement a specific protocol.

In [425]:
from nutree import Tree

tree = Tree("Hello")
tree.add("N").add("u").up(2).add("T").add("r").up().add("ee")
tree.print()

Tree<'Hello'>
├── 'N'
│   ╰── 'u'
╰── 'T'
    ├── 'r'
    ╰── 'ee'


Strings can be directly added to a tree, but in a real-world scenario we want to 
handle ordinary objects:

## Setup some sample classes and objects
Let's define a simple class hierarchy

In [426]:
import uuid


class Department:
    def __init__(self, name: str):
        self.guid = uuid.uuid4()
        self.name = name

    def __str__(self):
        return f"Department<{self.name}>"


class Person:
    def __init__(self, name: str, age: int):
        self.guid = uuid.uuid4()
        self.name = name
        self.age = age

    def __str__(self):
        return f"Person<{self.name} ({self.age})>"

and create some instances

In [427]:
development_dep = Department("Development")
test__dep = Department("Test")
marketing_dep = Department("Marketing")

alice = Person("Alice", 25)
bob = Person("Bob", 35)
claire = Person("Claire", 45)
dave = Person("Dave", 55)

Now that we have a bunch of instances, let's organize these objects in a 
hierarchical structure using _nutree_:

In [428]:
from nutree import Tree

tree = Tree("Organization")

dev_node = tree.add(development_dep)
test_node = dev_node.add(test__dep)
mkt_node = tree.add(marketing_dep)

tree.add(alice)
dev_node.add(bob)
test_node.add(claire)
mkt_node.add(dave)

tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x105356690>
│   ├── <__main__.Department object at 0x105356390>
│   │   ╰── <__main__.Person object at 0x10503c8c0>
│   ╰── <__main__.Person object at 0x10503d700>
├── <__main__.Department object at 0x10503dc70>
│   ╰── <__main__.Person object at 0x10503d430>
╰── <__main__.Person object at 0x10503d6a0>


Tree nodes store a reference to the object in the `node.data` attribute.

The nodes are formatted for display by the object's  `__repr__` implementation 
by default. <br>
We can overide this by passing an 
[f-string](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals) 
as `repr` argument. <br>
For example `"{node.data}"` will use the data instances `__str__` method instead:

In [429]:
tree.print(repr="{node.data}")

Tree<'Organization'>
├── Department<Development>
│   ├── Department<Test>
│   │   ╰── Person<Claire (45)>
│   ╰── Person<Bob (35)>
├── Department<Marketing>
│   ╰── Person<Dave (55)>
╰── Person<Alice (25)>


## Access Nodes
We can use the index syntax to get the node object for a given data object:

In [ ]:
tree[alice]

Node<'Person<Alice (25)>', data_id=273694058>

Note that we passed `alice` as index, which is an instance of `Person`, and received an instance of the `Node` container:

In [431]:
from nutree import Node

assert isinstance(tree[alice], Node)
assert tree[alice].data is alice, "nodes store objects in data attribute"

There are other other search methods as well

In [432]:
tree.find_all(lambda node: "i" in str(node.data))

[]

### Control the `data_id`

If the object instances have a natural attribute that identifies them, we can use
it instead of the default `hash()`. <br>
This improves readability and helps to (de)serialize:

In [433]:
tree_2 = Tree("Organization", calc_data_id=lambda tree, data: str(data.guid))
dep_node_2 = tree_2.add(development_dep)
dep_node_2.add(bob)
tree_2.print(repr="{node}")

Tree<'Organization'>
╰── Node<'Department<Development>', data_id=866c7479-c7ea-4702-aa32-88d380089c3c>
    ╰── Node<'Person<Bob (35)>', data_id=6d082e20-ff9e-44cd-afe7-8669cf0f8eec>


now we could also search by the guid, for example

In [434]:
tree_2.find(data_id=str(bob.guid))

Node<'Person<Bob (35)>', data_id=6d082e20-ff9e-44cd-afe7-8669cf0f8eec>

## Iteration and Searching

There are multiple methods to iterate the tree.

In [435]:
res = []
for node in tree:  # depth-first, pre-orde traversal
    res.append(node.data.name)
print(res)

['Development', 'Test', 'Claire', 'Bob', 'Marketing', 'Dave', 'Alice']


In [436]:
from nutree import IterMethod

res = []
for node in tree.iterator(method=IterMethod.POST_ORDER):
    res.append(node.data.name)
print(res)

['Claire', 'Test', 'Bob', 'Development', 'Dave', 'Marketing', 'Alice']


In [437]:
tree.visit(lambda node, memo: print(node.data.name), method=IterMethod.LEVEL_ORDER)

Development
Marketing
Alice
Test
Bob
Dave
Claire


The above traversal methods are also available for single nodes:

In [438]:
res = [node.data.name for node in dev_node]
print(res)

['Test', 'Claire', 'Bob']


## Filter

In [439]:
tree_copy = tree.copy(predicate=lambda node: isinstance(node.data, Department))
tree_copy.print(repr="{node}")

Tree<"Copy of Tree<'Organization'>">
├── Node<'Department<Development>', data_id=273897065>
│   ╰── Node<'Department<Test>', data_id=273897017>
╰── Node<'Department<Marketing>', data_id=273694151>


## Mutation

In [440]:
bob_node = tree[bob]
# bob_node.move_to(marketing_dep_node)
tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x105356690>
│   ├── <__main__.Department object at 0x105356390>
│   │   ╰── <__main__.Person object at 0x10503c8c0>
│   ╰── <__main__.Person object at 0x10503d700>
├── <__main__.Department object at 0x10503dc70>
│   ╰── <__main__.Person object at 0x10503d430>
╰── <__main__.Person object at 0x10503d6a0>


## Data IDs and Clones

In [441]:
tree.print(repr="{node}", title=False)

Node<'Department<Development>', data_id=273897065>
├── Node<'Department<Test>', data_id=273897017>
│   ╰── Node<'Person<Claire (45)>', data_id=273693836>
╰── Node<'Person<Bob (35)>', data_id=273694064>
Node<'Department<Marketing>', data_id=273694151>
╰── Node<'Person<Dave (55)>', data_id=273694019>
Node<'Person<Alice (25)>', data_id=273694058>


## Special Data Types
### Plain Strings

We can add simple string objects the same way as any other object

In [442]:
tree_str = Tree()
a = tree_str.add("A")
a.add("a1")
a.add("a2")
tree_str.add("B")
tree_str.print()

Tree<'4379102432'>
├── 'A'
│   ├── 'a1'
│   ╰── 'a2'
╰── 'B'


### Dictionaries

We cannot add Python `dict` objects to a tree, because nutree cannot derive
a *data_id* for unhashable types. <br>
A a workaround, we can wrap it inside `DictWrapper` objects:

In [443]:
from nutree import DictWrapper, Tree

d = {"title": "foo", "id": 1}

tree = Tree()
tree.add("A").up().add("B")
tree["A"].add(DictWrapper(d))
tree["B"].add(DictWrapper(d))
tree.print(repr="{node}")
# tree.find(d)

Tree<'4379111888'>
├── Node<'A', data_id=8244009354550040280>
│   ╰── Node<"DictWrapper<{'title': 'foo', 'id': 1}>", data_id=4382092096>
╰── Node<'B', data_id=6774184725397424230>
    ╰── Node<"DictWrapper<{'title': 'foo', 'id': 1}>", data_id=4382092096>


## Serialization

In [444]:
tree.to_dict_list()
# tree.to_dict_list(mapper=lambda node, data: node.data.name)

[{'data': 'A',
  'children': [{'data': "DictWrapper<{'title': 'foo', 'id': 1}>"}]},
 {'data': 'B',
  'children': [{'data': "DictWrapper<{'title': 'foo', 'id': 1}>"}]}]

In [445]:
list(tree.to_list_iter())

[(0, 'A'), (1, {}), (0, 'B'), (3, 2)]

In [446]:
t = Tree._from_list([(0, "A"), (0, "B"), (1, "C"), (0, "D"), (3, "E")])
print(t.format())

Tree<'4379107184'>
├── 'A'
│   ╰── 'C'
│       ╰── 'E'
├── 'B'
╰── 'D'


```mermaid
graph LR;
    A--> B & C & D;
    B--> A & E;
    C--> A & E;
    D--> A & E;
    E--> B & C & D;
```

## Advanced

### Chaining

Some methods return a node instance, so we can chain calls. <br>
This allows for a more compact code and avoids some temporary variables:

In [447]:
Tree().add("A").add("a1").up().add("a2").up(2).add("B").tree.print(title=False)

'A'
├── 'a1'
╰── 'a2'
'B'
